# Advent of Code
Charel Felten

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Advent

In [3]:
df = pd.read_csv("inputs/1", header=None)

f = lambda w: (df.rolling(w).sum().dropna().diff().fillna(0) > 0).sum().loc[0]

print(f"Task 1: {f(1)}, Task 2: {f(3)}")

Task 1: 1266, Task 2: 1217


## 2. Advent

In [4]:
df = pd.read_csv("inputs/2", header=None, sep=" ", names=["direction", "distance"])

t1 = df.groupby("direction").sum().apply(lambda col: (col["down"] - col["up"]) * col["forward"]).loc["distance"]

h, d, a = 0,0,0
for _, (dir, dis) in df.iterrows():
    if dir == "down": a += dis
    if dir == "up": a -= dis
    if dir == "forward":
        h += dis
        d += a*dis
        
print(f"Task 1: {t1}, Task 2: {h * d}")

Task 1: 1882980, Task 2: 1971232560


## 3. Advent

In [8]:
df = pd.read_csv("inputs/3", header=None, sep=" ", dtype="str")

t1 = df\
    .apply(lambda r: list(r.iloc[0]), result_type="expand", axis=1)\
    .agg(["median", lambda c: int(not(c.median()))])\
    .apply(lambda c: 2**((len(df.iloc[0,0])-1)-c.name) * c)\
    .sum(axis=1).prod().astype(int)

t2 = df.apply(lambda r: list(r.iloc[0]), result_type="expand", axis=1).astype(int)

def x(t2, b):
    for c in t2: t2 = t2[t2[c] == t2.groupby(c).size().sort_index(ascending=b).sort_values(ascending=b).index[0]]
    return t2.apply(lambda c: 2**((len(df.iloc[0,0])-1)-c.name) * c).sum(axis=1).iloc[0]

t2 = x(t2, False) * x(t2, True)

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 4147524, Task 2: 3570354


## 4. Advent

In [188]:
inputs = pd.read_csv("inputs/4", header=None, nrows=1).to_numpy().reshape(-1)
boards = pd.read_csv("inputs/4", header=None, skiprows=[0], sep=r" +").to_numpy().reshape(-1, 5, 5)
bingos = np.zeros_like(boards, dtype=bool)

aw = {}
for inp in inputs:
    for match in np.argwhere(boards == inp):
        bingos[tuple(match)] = True
    
    hs = np.argwhere(bingos.sum(axis=1) == boards.shape[1])
    vs = np.argwhere(bingos.sum(axis=2) == boards.shape[1])
    
    nw = set(list(vs[:,0]) + list(hs[:,0]))

    if len(nw) == 1 and len(aw) == 0:
        w = nw.difference(aw).pop()
        t1 = inp * boards[w, ~bingos[w]].sum()

    if len(nw) == boards.shape[0]:
        l = nw.difference(aw).pop()
        t2 = inp * boards[l, ~bingos[l]].sum()
        break

    aw = nw

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 34506, Task 2: 7686


## 5. Advent

In [240]:
df = pd.read_csv("inputs/5", header=None, sep=r",| -> ", names=["x1", "y1", "x2", "y2"], engine="python")

M = np.zeros(shape=[2] + [df.max().max() + 1] * 2)

for _, (x1,y1,x2,y2) in df.iterrows():
    sx = 1 if x2-x1 > 0 else -1
    sy = 1 if y2-y1 > 0 else -1
    if x1 == x2 or y1 == y2:
        for x in range(x1, x2 + sx, sx):
            for y in range(y1, y2 + sy, sy):
                M[:,y,x] += 1
    else:
        for x,y in zip(range(x1, x2 + sx, sx), range(y1, y2 + sy, sy)):
            M[1,y,x] += 1

print(f"Task 1: {np.sum(M[0] > 1)}, Task 2: {np.sum(M[1] > 1)}")

Task 1: 8622, Task 2: 22037


In [245]:
from itertools import zip_longest as zipl

df = pd.read_csv("inputs/5", header=None, sep=r",| -> ", names=["x1", "y1", "x2", "y2"], engine="python")

M = np.zeros(shape=[2] + [df.max().max() + 1] * 2)

for _, (x1,y1,x2,y2) in df.iterrows():
    for x,y in zipl(range(x1, x2 + (1 if x2-x1 > 0 else -1), (1 if x2-x1 > 0 else -1)), range(y1, y2 + (1 if y2-y1 > 0 else -1), (1 if y2-y1 > 0 else -1))):
        M[:, y if y is not None else y1, x if x is not None else x1] += (x1 == x2 or y1 == y2, 1)

print(f"Task 1: {np.sum(M[0] > 1)}, Task 2: {np.sum(M[1] > 1)}")

Task 1: 8622, Task 2: 22037


## 6. Advent

In [2]:
s = np.array([list(pd.read_csv("inputs/6", header=None).to_numpy().reshape(-1)).count(i) for i in range(9)])

def f(D, s):
    for _ in range(D):
        s = np.roll(s, -1)
        s[6] += s[8]
    return sum(s)

print(f"Task 1: {f(80, s)}, Task 2: {f(256, s)}")

Task 1: 390923, Task 2: 1749945484935


In [32]:
# Source: https://old.reddit.com/r/adventofcode/comments/r9z49j/2021_day_6_solutions/hnfi3nf/

TM = np.matrix([
    [0,1,0,0,0,0,0,0,0],  # this is a transition matrix, so each row encode the prob of transition
    [0,0,1,0,0,0,0,0,0],  # e.g. this row (row 1) has a 1 at col 2, which indicates that there
    [0,0,0,1,0,0,0,0,0],  # is a 100% chance (probability 1) to move from age 2 (col 2) to age 1 (row 1)
    [0,0,0,0,1,0,0,0,0],  # and a 0% chance for all other transitions.
    [0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0,0],
    [1,0,0,0,0,0,0,1,0],  # Fish being reset move from age 0 (col 0) to age 6 (col 6), but so do also
    [0,0,0,0,0,0,0,0,1],  # fish of age 7 (col 7) who move to age 6 (row 6). So two `1`s in this row.
    [1,0,0,0,0,0,0,0,0],  # New fish born move from age 0 (col 0) moves to age 8 (row 8)
])

print(f"Task 1: {np.sum(np.array(TM ** 80)*s)}, Task 2: {np.sum(np.array(TM ** 256)*s)}")

TMTM = np.matrix([
    [0,0,1,0,0,0,0,0,0],  # doing matrix multiplcation, which is the same as power 2, we get the state
    [0,0,0,1,0,0,0,0,0],  # of the matrix after 2 timesteps. Again we can read the fish ages
    [0,0,0,0,1,0,0,0,0],  # e.g. a fish originally aged 4 years (col 4), has a 100% chance to be
    [0,0,0,0,0,1,0,0,0],  # age 2 (row 2) after 2 iterations, which is indeed true
    [0,0,0,0,0,0,1,0,0],
    [1,0,0,0,0,0,0,1,0],  # there are two fish in this row, meaning that there are now 2 fish that are age 5 (row 5)
    [0,1,0,0,0,0,0,0,1],  # after 2 iterations, with one of them starting at age 0 (col 0) (0 -reset-> 6 -> 5)
    [1,0,0,0,0,0,0,0,0],  # and the other one being of age 7 (col 7) and simply aging normally (7 -> 6 -> 5)
    [0,1,0,0,0,0,0,0,0]
])

assert(np.all(TMTM == TM @ TM))

Task 1: 390923, Task 2: 1749945484935


## 7. Advent

In [18]:
p = pd.read_csv("inputs/7", header=None).to_numpy().reshape(-1)

t1 = int(sum(abs(p - np.median(p))))

trinum = lambda n: n * (n+1) / 2  # computes the n-th triangular number (0-indexing hence n+1)
t2 = int(sum(trinum(abs(p - np.floor(np.mean(p))))))

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 345197, Task 2: 96361606


In [19]:
# Note: my t2 above does not work for the example input (if using `np.floor` twice), and when one uses `np.round`
# twice, then it does not work for the real input. t2a below works for both verions, but is less efficient.

t2a = int(min([sum(trinum(abs(p - i))) for i in range(min(p), max(p))]))
print(f"Task 2 alt: {t2a}")

Task 2 alt: 96361606


<!-- a = np.array([list(pd.read_csv("inputs/6", header=None).to_numpy().reshape(-1)).count(i) for i in range(9)]) -->



<!--  -->